TODO:
- Read from cleaned csv
- Do some lemmitization or stemming

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

### Load data

In [1]:
## TODO: read from cleaned csv
FILENAME = "fake_job_postings.csv"
INDEX = "job_id"
TARGET_VARIABLE = "fraudulent"

df = pd.read_csv(FILENAME)
df.set_index(INDEX, inplace=True)
df_cleaned = pd.DataFrame(df[TARGET_VARIABLE])
df.head()

NameError: name 'pd' is not defined

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17880 entries, 1 to 17880
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                17880 non-null  object
 1   location             17534 non-null  object
 2   department           6333 non-null   object
 3   salary_range         2868 non-null   object
 4   company_profile      14572 non-null  object
 5   description          17879 non-null  object
 6   requirements         15185 non-null  object
 7   benefits             10670 non-null  object
 8   telecommuting        17880 non-null  int64 
 9   has_company_logo     17880 non-null  int64 
 10  has_questions        17880 non-null  int64 
 11  employment_type      14409 non-null  object
 12  required_experience  10830 non-null  object
 13  required_education   9775 non-null   object
 14  industry             12977 non-null  object
 15  function             11425 non-null  object
 16  frau

### TF-IDF Feature Extraction from Text Data

In [4]:
text_columns = ["title", "company_profile", "description", "requirements", "benefits"]
MAX_FEATURES = 100
NGRAM_RANGE = (1,1)

concatenated_text = df \
    .fillna("") \
    .apply(lambda row: " ".join([row[col] for col in text_columns]), axis=1)

tf_idf = TfidfVectorizer(stop_words="english", ngram_range=NGRAM_RANGE, max_features=MAX_FEATURES)
tf_idf_features = tf_idf.fit_transform(concatenated_text)
tf_idf_feature_names = "tfidf_" + np.array(tf_idf.get_feature_names(), dtype=object)
df_cleaned.loc[:,tf_idf_feature_names] = tf_idf_features.toarray()
df_cleaned.head()

,fraudulent,tfidf_ability,tfidf_able,tfidf_amp,tfidf_applications,tfidf_based,tfidf_benefits,tfidf_best,tfidf_build,tfidf_building,...,tfidf_training,tfidf_understanding,tfidf_using,tfidf_want,tfidf_web,tfidf_work,tfidf_working,tfidf_world,tfidf_year,tfidf_years
job_id,,,,,,,,,,,,,,,,,,,,,
1,0,0.00000,0.000000,0.107551,0.000000,0.000000,0.000000,0.126954,0.000000,0.0,...,0.00000,0.000000,0.142301,0.0,0.000000,0.150401,0.200518,0.000000,0.000000,0.000000
2,0,0.04155,0.049455,0.079301,0.056875,0.082357,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.052462,0.0,0.050881,0.083172,0.221774,0.188496,0.000000,0.000000
3,0,0.00000,0.000000,0.091046,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.000000,0.0,0.000000,0.127321,0.000000,0.000000,0.000000,0.000000
4,0,0.04846,0.000000,0.000000,0.000000,0.000000,0.117728,0.000000,0.061338,0.0,...,0.00000,0.250764,0.000000,0.0,0.000000,0.097004,0.000000,0.219844,0.064424,0.041253
5,0,0.00000,0.105770,0.084800,0.000000,0.000000,0.107941,0.200197,0.000000,0.0,...,0.11001,0.000000,0.000000,0.0,0.000000,0.059293,0.079051,0.000000,0.000000,0.075647


### Encode Categorical data

In [5]:
categorical_columns = ["department", "telecommuting", "has_company_logo", "has_questions", "employment_type", "required_experience", "required_education", "industry", "function"]

for col in categorical_columns:
    le = preprocessing.LabelEncoder()
    df_cleaned[col] = le.fit_transform(df[col])

df_cleaned[categorical_columns].head()

,department,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
job_id,,,,,,,,,
1,758,0,1,0,2,4,13,131,22
2,1161,0,1,0,1,6,13,75,7
3,1337,0,1,0,5,7,13,131,37
4,1054,0,1,0,1,5,1,22,31
5,1337,0,1,1,1,5,1,51,16


### Export Cleaned Data

In [6]:
df_cleaned.to_csv("cleaned_" + FILENAME)